In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 15:04:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

In [5]:
!ls -lh fhvhv_tripdata_2021-06.csv.gz

-rw-rw-r-- 1 bartoszsklodowski bartoszsklodowski 168M Mar  3 15:02 fhvhv_tripdata_2021-06.csv.gz


In [9]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", True) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

df = df.repartition(12)

df.write.parquet('data/pq/fhvhv/2021/06/')

23/03/03 15:20:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/03/03 15:20:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [10]:
!ls -lh data/pq/fhvhv/2021/06/

total 284M
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24M Mar  3 15:20 part-00000-f59d8fc9-d00c-40a0-8b69-039fbb05c678-c000.snappy.parquet
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24M Mar  3 15:20 part-00001-f59d8fc9-d00c-40a0-8b69-039fbb05c678-c000.snappy.parquet
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24M Mar  3 15:20 part-00002-f59d8fc9-d00c-40a0-8b69-039fbb05c678-c000.snappy.parquet
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24M Mar  3 15:20 part-00003-f59d8fc9-d00c-40a0-8b69-039fbb05c678-c000.snappy.parquet
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24M Mar  3 15:20 part-00004-f59d8fc9-d00c-40a0-8b69-039fbb05c678-c000.snappy.parquet
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24M Mar  3 15:20 part-00005-f59d8fc9-d00c-40a0-8b69-039fbb05c678-c000.snappy.parquet
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24M Mar  3 15:20 part-00006-f59d8fc9-d00c-40a0-8b69-039fbb05c678-c000.snappy.parquet
-rw-r--r-- 1 bartoszsklodowski bartoszsklodowski 24

In [11]:
df = spark.read.parquet('data/pq/fhvhv/2021/06/')

In [12]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

**Q3**: How many taxi trips were there on June 15??

In [14]:
from pyspark.sql import functions as F

In [15]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

**Q4**: Longest trip for each day

In [22]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [24]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long')) \
    .show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|  duration|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+
|              B02877|2021-06-11 12:28:02|2021-06-11 12:36:01|         230|          43|      N|                B02877|1623407761|
|              B02765|2021-06-13 08:46:22|2021-06-13 09:03:00|          37|          61|      N|                B02765|1623567780|
|              B02510|2021-06-28 18:39:27|2021-06-28 18:50:53|         231|          79|      N|                  null|1624899053|
|              B02869|2021-06-05 18:15:05|2021-06-05 18:28:32|         244|         127|      N|                B02869|1622910512|
|              B02872|2021-06-06 19:15:38|2021-06-06 19:26:58|          82|        

In [25]:
df \
    .withColumn('duration', (df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long'))/3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+------------------+
|pickup_date|     max(duration)|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
+-----------+------------------+



**Q6**: What is the name of the most frequent pickup location zone?

In [26]:
zones = spark.read \
    .option("header", "true") \
    .option("inferSchema", True) \
    .csv('taxi+_zone_lookup.csv')


In [28]:
zones.createOrReplaceTempView('zones')

In [29]:
df.createOrReplaceTempView('fhvhv_2021_06')

In [31]:
spark.sql("""
SELECT
    pul.Zone AS pul_zone,
    COUNT(1)
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+-------------------+--------+
|           pul_zone|count(1)|
+-------------------+--------+
|Crown Heights North|  231279|
|       East Village|  221244|
|        JFK Airport|  188867|
|     Bushwick South|  187929|
|      East New York|  186780|
+-------------------+--------+

